<a href="https://colab.research.google.com/github/Adesh4477/NEW-MODEL/blob/master/XLNet%20NLP%20task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data=pd.read_csv("/content/labeled_data.csv")
tweet=[i for i in data["tweet"]]

In [ ]:
import numpy as np

In [ ]:
label=['hate_speech', 'offensive_language', 'neither']

In [ ]:
class_label={i :label[i] for i in data["class"].unique()}

In [ ]:
class_label

{2: 'neither', 1: 'offensive_language', 0: 'hate_speech'}

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(tweet[:1000], np.array(data["class"])[:1000], test_size=0.2, random_state=42)

# Split the training data into training and validation sets (80-20 split)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute classification report
    class_report = classification_report(labels, predictions, output_dict=True)

    # Compute confusion matrix
    conf_matrix = confusion_matrix(labels, predictions).tolist() # Convert to a list to make it JSON serializable

    # Combine all the metrics into a dictionary
    metrics = {'accuracy': accuracy, 'classification_report': class_report, 'confusion_matrix': conf_matrix}

    return metrics

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch

In [ ]:
pip install sentencepiece

In [ ]:
# Load the tokenizer for XLNet
# This tokenizer will be used to convert text into tokens that the model can understand.
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Define the model for sequence classification
# Here, we're specifying the number of labels to classify.
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Function to tokenize the data
# This converts raw text data into a format that can be fed into the XLNet model.
def tokenize_data(data, labels):
    encodings = tokenizer(data, truncation=True, padding=True)
    return {"input_ids": encodings['input_ids'], "attention_mask": encodings['attention_mask'], "labels": labels}


# Custom Dataset class for handling the data
# This class will be used by PyTorch DataLoader to feed data into the model.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


# Preparing the dataset
# Converting the training and validation texts into lists.
train_texts = list(train_texts)
valid_texts = list(valid_texts)

# Tokenizing the training and validation data.
train_data = tokenize_data(train_texts, train_labels)
valid_data = tokenize_data(valid_texts, valid_labels)

# Creating instances of the custom dataset with the tokenized data.
train_dataset = CustomDataset(train_data)
valid_dataset = CustomDataset(valid_data)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
training_args = TrainingArguments(
    output_dir='./output', # Directory for storing outputs
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs', # Directory for storing logs
    report_to=[] # Disable all logging
)

# Initialize the Trainer
# This will handle the training and evaluation process.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

# Start training the model
# This step will train the model using the provided data and settings.
trainer.train()

Step,Training Loss


TrainOutput(global_step=80, training_loss=0.7585527896881104, metrics={'train_runtime': 1551.3952, 'train_samples_per_second': 0.413, 'train_steps_per_second': 0.052, 'total_flos': 56620425081600.0, 'train_loss': 0.7585527896881104, 'epoch': 1.0})

In [ ]:
test_texts = list(test_texts)
test_data = tokenize_data(test_texts, test_labels)
test_dataset = CustomDataset(test_data)

# Evaluate the model on the test dataset
results = trainer.evaluate(test_dataset)

print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.6248934864997864, 'eval_accuracy': 0.755, 'eval_classification_report': {'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 7}, '1': {'precision': 0.755, 'recall': 1.0, 'f1-score': 0.8603988603988605, 'support': 151}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 42}, 'accuracy': 0.755, 'macro avg': {'precision': 0.25166666666666665, 'recall': 0.3333333333333333, 'f1-score': 0.2867996201329535, 'support': 200}, 'weighted avg': {'precision': 0.570025, 'recall': 0.755, 'f1-score': 0.6496011396011397, 'support': 200}}, 'eval_confusion_matrix': [[0, 7, 0], [0, 151, 0], [0, 42, 0]], 'eval_runtime': 204.2781, 'eval_samples_per_second': 0.979, 'eval_steps_per_second': 0.122, 'epoch': 1.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Creating rows for each category and the averages
import plotly.graph_objects as go
category_names = {
    '2': 'neither', '1': 'offensive_language', '0': 'hate_speech'
}
classification_report = results['eval_classification_report']
rows = []
for key, value in classification_report.items():
    if key not in ['accuracy', 'macro avg', 'weighted avg']:
        category_name = category_names[key]
        row = [category_name] + [value['precision'], value['recall'], value['f1-score'], value['support']]
        rows.append(row)

# Adding averages
for avg in ['macro avg', 'weighted avg']:
    row = [avg] + [classification_report[avg][metric] for metric in ['precision', 'recall', 'f1-score', 'support']]
    rows.append(row)

# Convert rows from zip object to a list of lists
cell_values = list(map(list, zip(*rows)))

# Creating the table
fig = go.Figure(data=[go.Table(
    header=dict(values=['C'0': 'ethnicity/race',
    '1': 'gender/sexual',
    '2': '',ategory', 'Precision', 'Recall', 'F1-Score', 'Support'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=cell_values,
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.show()